# BTE -- Question #2 -- Use Case Workflow

## 0. Uploads, Functions, and Parameters

In [1]:
# Import pandas and biothings explorers modules
import pandas as pd
from biothings_explorer.user_query_dispatcher import FindConnection
from biothings_explorer.hint import Hint
ht = Hint()

In [2]:
# set list constant that represents every node type available in BTE
ALL_NODE_TYPES = ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
                'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
                'Pathway', 'AnatomicalEntity', 'PhenotypicFeature']

In [3]:
# predict_many funciton will be used to run many BTE queries and return results as a single table
def predict_many(input_object_list, output_type_list, intermediate_node_list = ''):
    df_list = []
    for input_object in input_object_list: 
        for output_type in output_type_list: 
            if(len(intermediate_node_list) > 0):
                for inter in intermediate_node_list:
                    try: 
                        print("Running: " + input_object['name'] + ' --> intermediate type ' + inter + ' --> output type ' + output_type )
                        fc = FindConnection(input_obj=input_object, output_obj=output_type, intermediate_nodes=[inter])
                        fc.connect(verbose=False)
                        df = fc.display_table_view()
                        rows = df.shape[0]
                        if(rows > 0):
                            df_list.append(df)
                    except:
                        print(input_object['name'] + ' --> intermediate type ' + inter + ' --> output type ' + output_type + ' FAILED')
            else:
                try:
                    print("Running: " + input_object['name'] + ' --> output type ' + output_type )
                    fc = FindConnection(input_obj=input_object, output_obj=output_type, intermediate_nodes=None)
                    fc.connect(verbose=False)
                    df = fc.display_table_view()
                    rows = df.shape[0]
                    if(rows > 0):
                        df_list.append(df)
                except:
                    print(input_object['name'] + ' --> output type ' + output_type + ' FAILED')

    if(len(df_list) > 0):
        return pd.concat(df_list)
    else:
        return None

In [4]:
# max_one_step_genes represents the number of genes returned from direct disease -> gene query 
# that will be included in results. Genes with most occurrences will be included over those with less
max_one_step_genes = 50

# max_two_step_genes represents the number of genes returned from disease -> intermediate node -> gene query 
max_two_step_genes = 100

# set disease name
disease_name = 'thrombophilia due to activated protein C resistance'

# set disease output files
disease_csv_file = 'Dravet_Syndrome_BTE_2020_09_09.csv'
disease_symptoms_csv = 'Dravet_Syndrome_Symptoms_2020_09_09.csv'

## 1. Get Disease Symptoms and Symptom Information 

### 1.1 Get Disease

In [5]:
# run hint query to get disease input
disease = ht.query(disease_name)['Disease'][0]
print(disease)

{'MONDO': 'MONDO:0008560', 'UMLS': 'C1861171', 'name': 'thrombophilia due to activated protein C resistance', 'MESH': 'D020016', 'OMIM': '188055', 'primary': {'identifier': 'MONDO', 'cls': 'Disease', 'value': 'MONDO:0008560'}, 'display': 'MONDO(MONDO:0008560) OMIM(188055) UMLS(C1861171) MESH(D020016) name(thrombophilia due to activated protein C resistance)', 'type': 'Disease'}


### 1.2 Get 'PhenotypicFeatures' Related to Disease

In [6]:
# get phenotypes (signs and symptoms) related to diesase
fc = FindConnection(input_obj=disease, output_obj='PhenotypicFeature', intermediate_nodes=None)
fc.connect(verbose=False)
disease_to_phenotypicFeature = fc.display_table_view()
disease_to_phenotypicFeature

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,output_type,output_name,output_id
0,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,None,BioLink API,None,PhenotypicFeature,BLOOD CLOT IN A DEEP VEIN,UMLS:C0149871
1,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,hpo,mydisease.info API,None,PhenotypicFeature,BLOOD CLOT IN A DEEP VEIN,UMLS:C0149871
2,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,None,BioLink API,None,PhenotypicFeature,ACTIVATED PROTEIN C RESISTANCE,UMLS:C0600433
3,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,hpo,mydisease.info API,None,PhenotypicFeature,ACTIVATED PROTEIN C RESISTANCE,UMLS:C0600433
4,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,None,BioLink API,None,PhenotypicFeature,ABNORMAL PARTIAL THROMBOPLASTIN TIME,UMLS:C0240671
5,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,hpo,mydisease.info API,None,PhenotypicFeature,ABNORMAL PARTIAL THROMBOPLASTIN TIME,UMLS:C0240671
6,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,None,BioLink API,None,PhenotypicFeature,BLOOD HYPERVISCOSITY,UMLS:C0398623
7,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,hpo,mydisease.info API,None,PhenotypicFeature,BLOOD HYPERVISCOSITY,UMLS:C0398623
8,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,None,BioLink API,None,PhenotypicFeature,PRE-ECLAMPSIA,UMLS:C0032914
9,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,hpo,mydisease.info API,None,PhenotypicFeature,PRE-ECLAMPSIA,UMLS:C0032914


In [7]:
# Print equivalent names for the disease input
print('Note: all equivalent names for the disease input are as follows:')
for name in fc.fc.display_node_info(disease_name)['equivalent_ids']['name']: print(name)

Note: all equivalent names for the disease input are as follows:
ACTIVATED PROTEIN C RESISTANCE
APC RESISTANCE
PCCF DEFICIENCY
PROC COFACTOR DEFICIENCY
RESISTANCE, APC
THPH2
THROMBOPHILIA 5
THROMBOPHILIA DUE TO ACTIVATED PROTEIN C RESISTANCE
THROMBOPHILIA DUE TO ACTIVATED PROTEIN C RESISTANCE; THPH2
THROMBOPHILIA DUE TO DEFICIENCY OF ACTIVATED PROTEIN C COFACTOR
THROMBOPHILIA DUE TO FACTOR 5 LEIDEN


In [8]:
# create dictionary of symptom HPIDs, nad symptom names (with synonyms)
symptom_dict = {}
for index, row in disease_to_phenotypicFeature.iterrows():
    output_name = disease_to_phenotypicFeature['output_name'][index]
    items = fc.fc.G[disease_name][output_name].values()
    for item in items: 
#         print(item)
        if('frequency' in item['info']):
            freq = [_item['info']['frequency'] for _item in fc.fc.G[disease_name][output_name].values() if "frequency" in _item["info"]][0][0]
            freq_value = ht.query(freq)['PhenotypicFeature'][0]['name']
        else: 
            freq_value = 'Unknown'
                    
        symptom_dict[fc.fc.display_node_info(output_name)['equivalent_ids']['HP'][0]] = {
            "names": fc.fc.display_node_info(output_name)['equivalent_ids']['name'],
            "frequency": freq_value,
        }
print(symptom_dict)

{'HP:0002625': {'names': ['BLOOD CLOT IN A DEEP VEIN', 'DEEP VEIN THROMBOSIS', 'DEEP VENOUS THROMBOSIS', 'MULTIPLE DEEP VENOUS THROMBOSIS'], 'frequency': 'Unknown'}, 'HP:0012175': {'names': ['ACTIVATED PROTEIN C RESISTANCE', 'RESISTANCE TO ACTIVATED PROTEIN C'], 'frequency': 'Unknown'}, 'HP:0003645': {'names': ['ABNORMAL PARTIAL THROMBOPLASTIN TIME', 'DELAYED THROMBOPLASTIN GENERATION', 'PARTIAL THROMBOPLASTIN TIME PROLONGED', 'PROLONGED ACTIVATED PARTIAL THROMBOPLASTIN TIME', 'PROLONGED PARTIAL THROMBOPLASTIN TIME', 'PROLONGED PTT'], 'frequency': 'Unknown'}, 'HP:0100724': {'names': ['BLOOD HYPERVISCOSITY', 'HYPERCOAGULABILITY', 'THROMBOPHILIA'], 'frequency': 'Unknown'}, 'HP:0100602': {'names': ['PRE-ECLAMPSIA', 'PREECLAMPSIA'], 'frequency': 'Unknown'}, 'HP:0000006': {'names': ['AUTOSOMAL DOMINANT', 'AUTOSOMAL DOMINANT FORM', 'AUTOSOMAL DOMINANT INHERITANCE', 'AUTOSOMAL DOMINANT TYPE'], 'frequency': 'Unknown'}}


In [9]:
# create list of HPIDs and Symptoms for later use
disease_symptom_hpids = list(symptom_dict.keys())
disease_symptoms = []
for entry in list(symptom_dict.values()): disease_symptoms = disease_symptoms + entry['names']


# create dict of HPIDs : symptoms for use in assembling final results
symptom_to_hpid_dict = {}
for key,value in symptom_dict.items(): 
    for name in value['names']:
        symptom_to_hpid_dict[name] = key

In [10]:
# add "edges out" counts from each phenotype to any node type, to get a rough estimate of how prevalent a phenotype is
phenotype_inputs = []
for hpid in disease_symptom_hpids: 
    try: 
        phenotype_input = ht.query(hpid)['PhenotypicFeature'][0]
        all_edges_out_df = predict_many([phenotype_input], ALL_NODE_TYPES)
        symptom_dict[hpid]['edges_out_count'] = all_edges_out_df.shape[0]
    except: 
        print(hpid + ' Failed')

print(symptom_dict)

Running: Deep venous thrombosis --> output type Gene
Running: Deep venous thrombosis --> output type SequenceVariant
Running: Deep venous thrombosis --> output type ChemicalSubstance
Running: Deep venous thrombosis --> output type Disease
Running: Deep venous thrombosis --> output type MolecularActivity
Running: Deep venous thrombosis --> output type BiologicalProcess
Running: Deep venous thrombosis --> output type CellularComponent
Running: Deep venous thrombosis --> output type Pathway
Running: Deep venous thrombosis --> output type AnatomicalEntity
Running: Deep venous thrombosis --> output type PhenotypicFeature
Running: Resistance to activated protein C --> output type Gene
Running: Resistance to activated protein C --> output type SequenceVariant
Running: Resistance to activated protein C --> output type ChemicalSubstance
Running: Resistance to activated protein C --> output type Disease
Running: Resistance to activated protein C --> output type MolecularActivity
Running: Resista

In [11]:
# convert symptom dictionary to dataframe and sort by edges out and frequency 
disease_symptom_df = pd.DataFrame.from_dict(symptom_dict, orient='index').sort_values(by=['edges_out_count'])
disease_symptom_df = pd.concat([disease_symptom_df[disease_symptom_df["frequency"] == "Very frequent"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Frequent"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Occasional"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Rare"],
               disease_symptom_df[disease_symptom_df["frequency"] == "Unknown"]
              ])
disease_symptom_df

,names,frequency,edges_out_count
HP:0012175,"[ACTIVATED PROTEIN C RESISTANCE, RESISTANCE TO...",Unknown,4
HP:0100602,"[PRE-ECLAMPSIA, PREECLAMPSIA]",Unknown,36
HP:0002625,"[BLOOD CLOT IN A DEEP VEIN, DEEP VEIN THROMBOS...",Unknown,38
HP:0100724,"[BLOOD HYPERVISCOSITY, HYPERCOAGULABILITY, THR...",Unknown,47
HP:0003645,"[ABNORMAL PARTIAL THROMBOPLASTIN TIME, DELAYED...",Unknown,76
HP:0000006,"[AUTOSOMAL DOMINANT, AUTOSOMAL DOMINANT FORM, ...",Unknown,538


In [12]:
disease_symptom_df.to_csv(disease_symptoms_csv, index = True)

## 2. Get Genes Directly Related to Disease

In [13]:
# find genes directly related to disease
fc = FindConnection(input_obj=disease, output_obj='Gene', intermediate_nodes=None)
fc.connect(verbose=False)
disease_to_genes = fc.display_table_view()
disease_to_genes = disease_to_genes[~disease_to_genes['output_id'].str.contains('UMLS')]
disease_to_genes

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,output_type,output_name,output_id
20,ACTIVATED PROTEIN C RESISTANCE,Disease,disrupted_by,SEMMED,SEMMED Disease API,7745572,Gene,CDSN,NCBIGene:1041
21,ACTIVATED PROTEIN C RESISTANCE,Disease,affected_by,SEMMED,SEMMED Disease API,"11703344,16493484",Gene,CDSN,NCBIGene:1041
22,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,SEMMED,SEMMED Disease API,8834124,Gene,CDSN,NCBIGene:1041
23,ACTIVATED PROTEIN C RESISTANCE,Disease,caused_by,SEMMED,SEMMED Disease API,"19036061,19764414,19803829",Gene,CDSN,NCBIGene:1041
24,ACTIVATED PROTEIN C RESISTANCE,Disease,disrupted_by,SEMMED,SEMMED Disease API,12624624,Gene,CD82,NCBIGene:3732
25,ACTIVATED PROTEIN C RESISTANCE,Disease,affected_by,SEMMED,SEMMED Disease API,11703344,Gene,TFPI,NCBIGene:7035
26,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,disgenet,mydisease.info API,None,Gene,TFPI,NCBIGene:7035
27,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,pharos,Automat PHAROS API,None,Gene,TFPI,NCBIGene:7035
29,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,SEMMED,SEMMED Disease API,7631299,Gene,KLK3,NCBIGene:354
30,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,SEMMED,SEMMED Disease API,20161734,Gene,SERPINE1,NCBIGene:5054


In [14]:
# create dictionary of gene reults whereby each Gene has values for the number of occurrences of the gene in results
# and the number of publications connecting the disease -> gene directly
disease_to_gene_results = {}
disease_to_gene_genes = list(disease_to_genes["output_name"]) # create list of genes
disease_to_gene_genes = list(dict.fromkeys(disease_to_gene_genes))  # remove duplicates

for gene in disease_to_gene_genes: 
    disease_to_gene_results[gene] = {
        'gene_count' : 0,
        'publications': []
    }

for index, row in disease_to_genes.iterrows():
    disease_to_gene_results[row['output_name']]['gene_count'] = disease_to_gene_results[row['output_name']]['gene_count'] + 1
    if(row['pred1_pubmed'] != None): 
        disease_to_gene_results[row['output_name']]['publications'] = disease_to_gene_results[row['output_name']]['publications'] + row['pred1_pubmed'].split(",")

disease_to_gene_results = dict(sorted(disease_to_gene_results.items(), key = lambda x: x[1]['gene_count'], reverse = True))

for key,value in disease_to_gene_results.items(): 
    disease_to_gene_results[key]['publications'] = list(dict.fromkeys(disease_to_gene_results[key]['publications']))
    
    
disease_to_gene_results


{'CDSN': {'gene_count': 4,
  'publications': ['7745572',
   '11703344',
   '16493484',
   '8834124',
   '19036061',
   '19764414',
   '19803829']},
 'TFPI': {'gene_count': 3, 'publications': ['11703344']},
 'F5': {'gene_count': 3, 'publications': []},
 'SSB': {'gene_count': 2, 'publications': ['21846908', '16315641']},
 'PROS1': {'gene_count': 2, 'publications': []},
 'CD82': {'gene_count': 1, 'publications': ['12624624']},
 'KLK3': {'gene_count': 1, 'publications': ['7631299']},
 'SERPINE1': {'gene_count': 1, 'publications': ['20161734']},
 'SERPINB2': {'gene_count': 1, 'publications': ['20161734']},
 'JAK2': {'gene_count': 1, 'publications': ['18768782']},
 'TWIST1': {'gene_count': 1, 'publications': ['21914670']},
 'TUBB4B': {'gene_count': 1, 'publications': ['16315641']},
 'NEUROD1': {'gene_count': 1, 'publications': ['16315641']},
 'CP': {'gene_count': 1, 'publications': ['9493610']},
 'GPI': {'gene_count': 1, 'publications': ['15637132']},
 'F8': {'gene_count': 1, 'publications':

## 3. Get Top Genes Related to Disease through 1 Intermediate Node

In [15]:
# get results for disease -> any node type -> gene
disease_to_all_nodes_to_genes = predict_many([disease],['Gene'], ALL_NODE_TYPES)
disease_to_all_nodes_to_genes.head() 

Running: thrombophilia due to activated protein C resistance --> intermediate type Gene --> output type Gene
Running: thrombophilia due to activated protein C resistance --> intermediate type SequenceVariant --> output type Gene
Running: thrombophilia due to activated protein C resistance --> intermediate type ChemicalSubstance --> output type Gene
API 6.1 pharos failed
Running: thrombophilia due to activated protein C resistance --> intermediate type Disease --> output type Gene
Running: thrombophilia due to activated protein C resistance --> intermediate type MolecularActivity --> output type Gene
Running: thrombophilia due to activated protein C resistance --> intermediate type BiologicalProcess --> output type Gene
Running: thrombophilia due to activated protein C resistance --> intermediate type CellularComponent --> output type Gene
Running: thrombophilia due to activated protein C resistance --> intermediate type Pathway --> output type Gene
Running: thrombophilia due to activat

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,output_type,output_name,output_id
0,ACTIVATED PROTEIN C RESISTANCE,Disease,treated_by,SEMMED,SEMMED Disease API,9974416,Gene,C1333075,UMLS:C1333075,positively_regulates,SEMMED,SEMMED Gene API,10048117,Gene,C0128902,UMLS:C0128902
1,ACTIVATED PROTEIN C RESISTANCE,Disease,treated_by,SEMMED,SEMMED Disease API,9974416,Gene,C1333075,UMLS:C1333075,physically_interacts_with,SEMMED,SEMMED Gene API,27210873,Gene,F9,NCBIGene:2158
2,ACTIVATED PROTEIN C RESISTANCE,Disease,disrupted_by,SEMMED,SEMMED Disease API,14515684,Gene,C2985367,UMLS:C2985367,positively_regulates,SEMMED,SEMMED Gene API,7541242,Gene,F9,NCBIGene:2158
3,ACTIVATED PROTEIN C RESISTANCE,Disease,affected_by,SEMMED,SEMMED Disease API,"16879211,17549295,8618815,8701401,8956934",Gene,C2985367,UMLS:C2985367,positively_regulates,SEMMED,SEMMED Gene API,7541242,Gene,F9,NCBIGene:2158
4,ACTIVATED PROTEIN C RESISTANCE,Disease,related_to,SEMMED,SEMMED Disease API,28717431,Gene,C2985367,UMLS:C2985367,positively_regulates,SEMMED,SEMMED Gene API,7541242,Gene,F9,NCBIGene:2158


In [16]:
# Save
%store disease_to_all_nodes_to_genes
%store -r disease_to_all_nodes_to_genes

Stored 'disease_to_all_nodes_to_genes' (DataFrame)


In [17]:
# remove entries with symptoms as intermediates
indices_with_symptom_intermediates = [i for i, val in enumerate(list(disease_to_all_nodes_to_genes['node1_name'])) if val in disease_symptoms]
disease_to_all_nodes_to_genes = disease_to_all_nodes_to_genes.drop( disease_to_all_nodes_to_genes.index[indices_with_symptom_intermediates])
# remove UMLS entries - not totally gene specific
disease_to_all_nodes_to_genes = disease_to_all_nodes_to_genes[~disease_to_all_nodes_to_genes['output_id'].str.contains('UMLS')]
disease_to_all_nodes_to_genes.head()


,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,output_type,output_name,output_id
1663,ACTIVATED PROTEIN C RESISTANCE,Disease,disrupted_by,SEMMED,SEMMED Disease API,12624624,Gene,C1705784,UMLS:C1705784,negatively_regulates,SEMMED,SEMMED Gene API,27041584,Gene,U2AF2,NCBIGene:11338
1664,ACTIVATED PROTEIN C RESISTANCE,Disease,disrupted_by,SEMMED,SEMMED Disease API,12624624,Gene,C1705784,UMLS:C1705784,physically_interacts_with,SEMMED,SEMMED Gene API,27041584,Gene,U2AF2,NCBIGene:11338
1665,ACTIVATED PROTEIN C RESISTANCE,Disease,caused_by,SEMMED,SEMMED Disease API,9405896,Gene,C0017337,UMLS:C0017337,negatively_regulates,SEMMED,SEMMED Gene API,23152763,Gene,U2AF2,NCBIGene:11338
1666,ACTIVATED PROTEIN C RESISTANCE,Disease,disrupted_by,SEMMED,SEMMED Disease API,12624624,Gene,CD82,NCBIGene:3732,negatively_regulates,SEMMED,SEMMED Gene API,27041584,Gene,U2AF2,NCBIGene:11338
1667,ACTIVATED PROTEIN C RESISTANCE,Disease,disrupted_by,SEMMED,SEMMED Disease API,12624624,Gene,CD82,NCBIGene:3732,physically_interacts_with,SEMMED,SEMMED Gene API,27041584,Gene,U2AF2,NCBIGene:11338


In [18]:
# create dictionary of gene reults whereby each Gene has values for the number of occurrences of the gene in results
# and the number of publications connecting the disease -> intermediates -> gene
disease_to_all_nodes_to_genes_results = {}
disease_to_all_nodes_to_genes_genes = list(disease_to_all_nodes_to_genes["output_name"]) # create list of genes
disease_to_all_nodes_to_genes_genes = list(dict.fromkeys(disease_to_all_nodes_to_genes_genes))  # remove duplicates

for gene in disease_to_all_nodes_to_genes_genes: 
    disease_to_all_nodes_to_genes_results[gene] = {
        'gene_count' : 0,
        'publications': []
    }

for index, row in disease_to_all_nodes_to_genes.iterrows():
    disease_to_all_nodes_to_genes_results[row['output_name']]['gene_count'] = disease_to_all_nodes_to_genes_results[row['output_name']]['gene_count'] + 1
    if(row['pred1_pubmed'] != None): 
        disease_to_all_nodes_to_genes_results[row['output_name']]['publications'] = disease_to_all_nodes_to_genes_results[row['output_name']]['publications'] + row['pred1_pubmed'].split(",")
    if(row['pred2_pubmed'] != None): 
        disease_to_all_nodes_to_genes_results[row['output_name']]['publications'] = disease_to_all_nodes_to_genes_results[row['output_name']]['publications'] + row['pred2_pubmed'].split(",")

disease_to_all_nodes_to_genes_results = dict(sorted(disease_to_all_nodes_to_genes_results.items(), key = lambda x: x[1]['gene_count'], reverse = True))
# disease_to_all_nodes_to_genes_results

for key,value in disease_to_all_nodes_to_genes_results.items(): 
    disease_to_all_nodes_to_genes_results[key]['publications'] = list(dict.fromkeys(disease_to_all_nodes_to_genes_results[key]['publications']))
    
        
# printing top 10   
print("Top 10 Gene Occurrences : ")
{A:N['gene_count'] for (A,N) in [x for x in disease_to_all_nodes_to_genes_results.items()][:10]}

Top 10 Gene Occurrences : 


{'TNF': 472,
 'VEGFA': 423,
 'AKT1': 313,
 'ESR1': 311,
 'INS': 283,
 'TP53': 268,
 'MMP9': 267,
 'TGFB1': 259,
 'EGFR': 255,
 'CAT': 255}

## 4. Determine Genes to Further Analyze 

In [19]:
# get list of genes based off of "max" gene parameters
disease_top_genes_list = ([A for (A,N) in [x for x in disease_to_gene_results.items()][:max_one_step_genes]]
    + [A for (A,N) in [x for x in disease_to_all_nodes_to_genes_results.items()][:max_two_step_genes]])

disease_top_genes_list = list(dict.fromkeys(disease_top_genes_list))
disease_top_genes_list

['CDSN',
 'TFPI',
 'F5',
 'SSB',
 'PROS1',
 'CD82',
 'KLK3',
 'SERPINE1',
 'SERPINB2',
 'JAK2',
 'TWIST1',
 'TUBB4B',
 'NEUROD1',
 'CP',
 'GPI',
 'F8',
 'PLAT',
 'PLG',
 'PROC',
 'LPA',
 'TNF',
 'VEGFA',
 'AKT1',
 'ESR1',
 'INS',
 'TP53',
 'MMP9',
 'TGFB1',
 'EGFR',
 'CAT',
 'MMP2',
 'LEP',
 'IL6',
 'CA2',
 'CAMP',
 'MTHFR',
 'CD4',
 'AR',
 'IFNA1',
 'MAPK1',
 'FAS',
 'PTH',
 'STAT3',
 'FOS',
 'APP',
 'CDKN1A',
 'ALB',
 'IL2',
 'VWF',
 'MPO',
 'TLR4',
 'CCL2',
 'POMC',
 'LPL',
 'MAPK8',
 'REN',
 'EGF',
 'LOX',
 'CISH',
 'CCND1',
 'IL10',
 'CD40',
 'APOB',
 'CRK',
 'ACE',
 'TH',
 'ESR2',
 'TIMP1',
 'BAX',
 'PGR',
 'WDTC1',
 'TCEAL1',
 'GRK2',
 'HSPA4',
 'ATM',
 'CD36',
 'PTGS2',
 'ERVK-10',
 'PIK3CA',
 'H3P16',
 'MTOR',
 'RHO',
 'ABCB1',
 'KIT',
 'PTEN',
 'PPARA',
 'CXCR4',
 'CDKN2A',
 'IGF1',
 'RELA',
 'ADAMTS13',
 'GABPA',
 'CD44',
 'TAT',
 'ANG',
 'PGF',
 'TTR',
 'MYC',
 'ITGB2',
 'BRCA1',
 'ADA',
 'RHOA',
 'STAT5A',
 'ITGAM',
 'TF',
 'EGR1',
 'SST',
 'MAPK3',
 'SIRT1']

## 5. Get Disease Symptoms related to Genes
Genes -> Symptoms, then filter based on disease symptoms

In [20]:
# get gene inputs through hint module
gene_inputs = []
for gene in disease_top_genes_list: 
    try: 
        gene_input = ht.query(gene)["Gene"][0]
        gene_inputs.append(gene_input)
    except: 
        print(gene + ' Failed')

print(gene_inputs)

APP Failed
[{'NCBIGene': '1041', 'name': 'corneodesmosin', 'SYMBOL': 'CDSN', 'UMLS': 'C1413299', 'HGNC': '1802', 'UNIPROTKB': 'Q15517', 'ENSEMBL': 'ENSG00000137197', 'primary': {'identifier': 'NCBIGene', 'cls': 'Gene', 'value': '1041'}, 'display': 'NCBIGene(1041) ENSEMBL(ENSG00000137197) HGNC(1802) UMLS(C1413299) UNIPROTKB(Q15517) SYMBOL(CDSN)', 'type': 'Gene'}, {'NCBIGene': '7035', 'name': 'tissue factor pathway inhibitor', 'SYMBOL': 'TFPI', 'UMLS': 'C1420705', 'HGNC': '11760', 'UNIPROTKB': 'P10646', 'ENSEMBL': 'ENSG00000003436', 'primary': {'identifier': 'NCBIGene', 'cls': 'Gene', 'value': '7035'}, 'display': 'NCBIGene(7035) ENSEMBL(ENSG00000003436) HGNC(11760) UMLS(C1420705) UNIPROTKB(P10646) SYMBOL(TFPI)', 'type': 'Gene'}, {'NCBIGene': '2153', 'name': 'coagulation factor V', 'SYMBOL': 'F5', 'UMLS': 'C1414509', 'HGNC': '3542', 'UNIPROTKB': 'P12259', 'ENSEMBL': 'ENSG00000198734', 'primary': {'identifier': 'NCBIGene', 'cls': 'Gene', 'value': '2153'}, 'display': 'NCBIGene(2153) ENSEMBL

In [21]:
# get genes to symptoms, which could be represented as a phenotypic feature, biological process, or diesase
genes_to_symptoms = predict_many(gene_inputs, ['PhenotypicFeature','BiologicalProcess','Disease'])
print(genes_to_symptoms.shape)
genes_to_symptoms.head()

Running: corneodesmosin --> output type PhenotypicFeature
Running: corneodesmosin --> output type BiologicalProcess
Running: corneodesmosin --> output type Disease
Running: tissue factor pathway inhibitor --> output type PhenotypicFeature
Running: tissue factor pathway inhibitor --> output type BiologicalProcess
Running: tissue factor pathway inhibitor --> output type Disease
Running: coagulation factor V --> output type PhenotypicFeature
Running: coagulation factor V --> output type BiologicalProcess
Running: coagulation factor V --> output type Disease
Running: small RNA binding exonuclease protection factor La --> output type PhenotypicFeature
Running: small RNA binding exonuclease protection factor La --> output type BiologicalProcess
Running: small RNA binding exonuclease protection factor La --> output type Disease
Running: protein S --> output type PhenotypicFeature
Running: protein S --> output type BiologicalProcess
Running: protein S --> output type Disease
Running: CD82 mole

Running: signal transducer and activator of transcription 3 --> output type BiologicalProcess
Running: signal transducer and activator of transcription 3 --> output type Disease
Running: Fos proto-oncogene, AP-1 transcription factor subunit --> output type PhenotypicFeature
Running: Fos proto-oncogene, AP-1 transcription factor subunit --> output type BiologicalProcess
Running: Fos proto-oncogene, AP-1 transcription factor subunit --> output type Disease
Running: cyclin dependent kinase inhibitor 1A --> output type PhenotypicFeature
Running: cyclin dependent kinase inhibitor 1A --> output type BiologicalProcess
Running: cyclin dependent kinase inhibitor 1A --> output type Disease
Running: albumin --> output type PhenotypicFeature
Running: albumin --> output type BiologicalProcess
Running: albumin --> output type Disease
Running: interleukin 2 --> output type PhenotypicFeature
Running: interleukin 2 --> output type BiologicalProcess
Running: interleukin 2 --> output type Disease
Running

Running: KIT proto-oncogene, receptor tyrosine kinase --> output type Disease
Running: phosphatase and tensin homolog --> output type PhenotypicFeature
Running: phosphatase and tensin homolog --> output type BiologicalProcess
Running: phosphatase and tensin homolog --> output type Disease
Running: peroxisome proliferator activated receptor alpha --> output type PhenotypicFeature
Running: peroxisome proliferator activated receptor alpha --> output type BiologicalProcess
Running: peroxisome proliferator activated receptor alpha --> output type Disease
Running: C-X-C motif chemokine receptor 4 --> output type PhenotypicFeature
Running: C-X-C motif chemokine receptor 4 --> output type BiologicalProcess
Running: C-X-C motif chemokine receptor 4 --> output type Disease
Running: cyclin dependent kinase inhibitor 2A --> output type PhenotypicFeature
Running: cyclin dependent kinase inhibitor 2A --> output type BiologicalProcess
Running: cyclin dependent kinase inhibitor 2A --> output type Dise

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,output_type,output_name,output_id
0,CDSN,Gene,related_to,None,BioLink API,None,PhenotypicFeature,DECREASED BODY HEIGHT,UMLS:C0349588
1,CDSN,Gene,related_to,None,BioLink API,None,PhenotypicFeature,BRITTLE HAIR,UMLS:C4072837
2,CDSN,Gene,related_to,None,BioLink API,None,PhenotypicFeature,DECREASED NUMBER OF SCALP FOLLICLES,UMLS:C1857042
3,CDSN,Gene,related_to,None,BioLink API,None,PhenotypicFeature,ASTHMA,UMLS:C0004096
4,CDSN,Gene,related_to,None,BioLink API,None,PhenotypicFeature,ERYTHEMA,UMLS:C0041834


In [22]:
# filter gene -> symptoms table to only include rows where symptoms match disease symptoms
indices_with_symptom_outputs = [i for i, val in enumerate([x.upper() for x in list(genes_to_symptoms['output_name'])]) if val in disease_symptoms]
relevant_genes_to_symptoms_df = genes_to_symptoms.iloc[indices_with_symptom_outputs]
relevant_genes_to_symptoms_df.head()

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,output_type,output_name,output_id
119,CDSN,Gene,disrupts,SEMMED,SEMMED Gene API,7745572,Disease,ACTIVATED PROTEIN C RESISTANCE,MONDO:MONDO:0008560
120,CDSN,Gene,causes,SEMMED,SEMMED Gene API,"19036061,19764414,19803829",Disease,ACTIVATED PROTEIN C RESISTANCE,MONDO:MONDO:0008560
121,CDSN,Gene,related_to,SEMMED,SEMMED Gene API,8834124,Disease,ACTIVATED PROTEIN C RESISTANCE,MONDO:MONDO:0008560
122,CDSN,Gene,affects,SEMMED,SEMMED Gene API,"11703344,16493484",Disease,ACTIVATED PROTEIN C RESISTANCE,MONDO:MONDO:0008560
151,CDSN,Gene,causes,SEMMED,SEMMED Gene API,2146503,Disease,DEEP VEIN THROMBOSIS,UMLS:C0149871


In [23]:
%store relevant_genes_to_symptoms_df
%store -r relevant_genes_to_symptoms_df

Stored 'relevant_genes_to_symptoms_df' (DataFrame)


In [24]:
# create dictionary to keep track of symptoms a gene is related to, and the number of publications relating the 
# gene to any of the disease symptoms
symptoms_results = {}
relevant_top_genes_list = list(dict.fromkeys(list(relevant_genes_to_symptoms_df["input"])))
for x in relevant_top_genes_list:
    symptoms_results[x] = {
        "related_symptoms" : [],
        "publications": []
    }
for index, row in relevant_genes_to_symptoms_df.iterrows():
    symptoms_results[row["input"]]["related_symptoms"].append(row["output_name"])
    if(row["pred1_pubmed"]):
        symptoms_results[row["input"]]["publications"] = symptoms_results[row["input"]]["publications"] + row["pred1_pubmed"].split(',')

for key,value in symptoms_results.items(): 
    symptoms_results[key]['publications'] = list(dict.fromkeys(symptoms_results[key]['publications']))
            
        
# print(symptoms_results)

## 6. Get Genes Edges Out Count

In [25]:
# get edges out count from genes to any node type to get rough estimate of how well researched a gene is
relevant_gene_inputs = []

for gene_input in gene_inputs: 
    if(gene_input['SYMBOL'] in relevant_top_genes_list):
        relevant_gene_inputs.append(gene_input)
        
all_gene_edges_out = predict_many(relevant_gene_inputs, ALL_NODE_TYPES)
edges_out_genes_list = list(all_gene_edges_out["input"])
gene_edges_out = {x:edges_out_genes_list.count(x) for x in edges_out_genes_list}
print(gene_edges_out)

Running: corneodesmosin --> output type Gene
Running: corneodesmosin --> output type SequenceVariant
Running: corneodesmosin --> output type ChemicalSubstance
Running: corneodesmosin --> output type Disease
Running: corneodesmosin --> output type MolecularActivity
Running: corneodesmosin --> output type BiologicalProcess
Running: corneodesmosin --> output type CellularComponent
Running: corneodesmosin --> output type Pathway
Running: corneodesmosin --> output type AnatomicalEntity
Running: corneodesmosin --> output type PhenotypicFeature
Running: tissue factor pathway inhibitor --> output type Gene
Running: tissue factor pathway inhibitor --> output type SequenceVariant
Running: tissue factor pathway inhibitor --> output type ChemicalSubstance
Running: tissue factor pathway inhibitor --> output type Disease
Running: tissue factor pathway inhibitor --> output type MolecularActivity
Running: tissue factor pathway inhibitor --> output type BiologicalProcess
Running: tissue factor pathway 

Running: neuronal differentiation 1 --> output type Pathway
Running: neuronal differentiation 1 --> output type AnatomicalEntity
Running: neuronal differentiation 1 --> output type PhenotypicFeature
Running: ceruloplasmin --> output type Gene
Running: ceruloplasmin --> output type SequenceVariant
Running: ceruloplasmin --> output type ChemicalSubstance
Running: ceruloplasmin --> output type Disease
Running: ceruloplasmin --> output type MolecularActivity
Running: ceruloplasmin --> output type BiologicalProcess
Running: ceruloplasmin --> output type CellularComponent
Running: ceruloplasmin --> output type Pathway
Running: ceruloplasmin --> output type AnatomicalEntity
Running: ceruloplasmin --> output type PhenotypicFeature
Running: glucose-6-phosphate isomerase --> output type Gene
Running: glucose-6-phosphate isomerase --> output type SequenceVariant
Running: glucose-6-phosphate isomerase --> output type ChemicalSubstance
Running: glucose-6-phosphate isomerase --> output type Disease


Running: methylenetetrahydrofolate reductase --> output type AnatomicalEntity
Running: methylenetetrahydrofolate reductase --> output type PhenotypicFeature
Running: interferon alpha 1 --> output type Gene
Running: interferon alpha 1 --> output type SequenceVariant
Running: interferon alpha 1 --> output type ChemicalSubstance
Running: interferon alpha 1 --> output type Disease
Running: interferon alpha 1 --> output type MolecularActivity
Running: interferon alpha 1 --> output type BiologicalProcess
Running: interferon alpha 1 --> output type CellularComponent
Running: interferon alpha 1 --> output type Pathway
Running: interferon alpha 1 --> output type AnatomicalEntity
Running: interferon alpha 1 --> output type PhenotypicFeature
Running: parathyroid hormone --> output type Gene
Running: parathyroid hormone --> output type SequenceVariant
Running: parathyroid hormone --> output type ChemicalSubstance
Running: parathyroid hormone --> output type Disease
Running: parathyroid hormone -->

In [26]:
%store gene_edges_out
%store -r gene_edges_out

Stored 'gene_edges_out' (dict)


## 7. Assemble Genes related to both Disease and Disease Symptoms

In [27]:
# assemble final dictionary that includes all metrics for gene's connections to disease (direct and through one node)
# as well as to disease symptoms
final_dict = {}

for x in relevant_top_genes_list:
    symptoms_to_hpids = [];
    for symptom in symptoms_results[x]['related_symptoms']: 
        symptoms_to_hpids.append(symptom_to_hpid_dict[symptom])
    unique_symptoms = list(dict.fromkeys(symptoms_to_hpids))
    final_dict[x] = {
        "disease_to_gene_occurrences" : disease_to_gene_results[x]['gene_count'] if x in disease_to_gene_results else 0,
        "disease_to_gene_pub_counts" : len(disease_to_gene_results[x]['publications']) if x in disease_to_gene_results else 0,
        "disease_to_int_to_gen_occurrences" : disease_to_all_nodes_to_genes_results[x]['gene_count'] if x in disease_to_all_nodes_to_genes_results else 0,
        "disease_to_int_to_gene_pubs" : len(disease_to_all_nodes_to_genes_results[x]['publications']) if x in disease_to_all_nodes_to_genes_results else 0,
        "disease_symtpoms_gene_related_to" : symptoms_results[x]['related_symptoms'],
        "disease_symtpoms_gene_related_to_count" : len(symptoms_results[x]['related_symptoms']),
        "unique_symptoms_count": len(unique_symptoms),
        "gene_to_symptoms_pub_counts" : len(symptoms_results[x]['publications']),
        "gene_edges_out": gene_edges_out[x]
    }


In [28]:
final_df = pd.DataFrame(final_dict).transpose()
final_df

,disease_to_gene_occurrences,disease_to_gene_pub_counts,disease_to_int_to_gen_occurrences,disease_to_int_to_gene_pubs,disease_symtpoms_gene_related_to,disease_symtpoms_gene_related_to_count,unique_symptoms_count,gene_to_symptoms_pub_counts,gene_edges_out
CDSN,4,7,227,560,"[ACTIVATED PROTEIN C RESISTANCE, ACTIVATED PRO...",8,2,13,960
TFPI,3,1,176,322,"[ACTIVATED PROTEIN C RESISTANCE, ACTIVATED PRO...",5,2,1,1275
F5,3,0,64,87,"[ABNORMAL PARTIAL THROMBOPLASTIN TIME, BLOOD C...",9,5,9,1001
SSB,2,2,104,208,"[ACTIVATED PROTEIN C RESISTANCE, ACTIVATED PRO...",2,1,2,1380
PROS1,2,0,73,126,"[BLOOD HYPERVISCOSITY, BLOOD CLOT IN A DEEP VE...",6,3,14,705
CD82,1,1,41,80,[ACTIVATED PROTEIN C RESISTANCE],1,1,1,1676
KLK3,1,1,144,483,"[DEEP VEIN THROMBOSIS, ACTIVATED PROTEIN C RES...",2,2,2,2156
SERPINE1,1,1,217,399,"[DEEP VEIN THROMBOSIS, ACTIVATED PROTEIN C RES...",2,2,2,1935
SERPINB2,1,1,170,332,[ACTIVATED PROTEIN C RESISTANCE],1,1,1,1297
JAK2,1,1,101,202,"[DEEP VEIN THROMBOSIS, DEEP VEIN THROMBOSIS, A...",3,2,2,2975


In [29]:
final_df.to_csv(disease_csv_file, index = True)

## 9. Look at pathways and biological processes

In [ ]:
# pathways
gene_to_pathways = predict_many(relevant_gene_inputs, ['Pathway'])
gene_to_pathway_results = {}
gene_to_pathway_genes = list(gene_to_pathways["output_name"]) # create list of genes
gene_to_pathway_genes = list(dict.fromkeys(gene_to_pathway_genes))  # remove duplicates

for gene in gene_to_pathway_genes: 
    gene_to_pathway_results[gene] = {
        'pathway_count' : 0,
        "genes_related" : []
    }

for index, row in gene_to_pathways.iterrows():
    gene_to_pathway_results[row['output_name']]['pathway_count'] = gene_to_pathway_results[row['output_name']]['pathway_count'] + 1
    gene_to_pathway_results[row['output_name']]['genes_related'].append(row['input'])
    

gene_to_pathway_results = dict(sorted(gene_to_pathway_results.items(), key = lambda x: x[1]['pathway_count'], reverse = True))

    
gene_to_pathway_results


In [ ]:
gene_to_bioprocess = predict_many(relevant_gene_inputs, ['BiologicalProcess'])
gene_to_bioprocess_results = {}
gene_to_bioprocess_genes = list(gene_to_bioprocess["output_name"]) # create list of genes
gene_to_bioprocess_genes = list(dict.fromkeys(gene_to_bioprocess_genes))  # remove duplicates

for gene in gene_to_bioprocess_genes: 
    gene_to_bioprocess_results[gene] = {
        'bioprocess_count' : 0,
        "genes_related" : []
    }

for index, row in gene_to_bioprocess.iterrows():
    gene_to_bioprocess_results[row['output_name']]['bioprocess_count'] = gene_to_bioprocess_results[row['output_name']]['bioprocess_count'] + 1
    gene_to_bioprocess_results[row['output_name']]['genes_related'].append(row['input'])
    
gene_to_bioprocess_results = dict(sorted(gene_to_bioprocess_results.items(), key = lambda x: x[1]['bioprocess_count'], reverse = True))

for key in gene_to_bioprocess_results.keys(): 
    if(('C0' in key) or ('C1' in key)): 
        try: 
            name = ht.query(key)['BiologicalProcess'][0]['name']
            gene_to_bioprocess_results[name] = gene_to_bioprocess_results[key]
            del gene_to_bioprocess_results[key]
        except: 
            pass

In [ ]:
gene_to_pathway_df = pd.DataFrame(gene_to_pathway_results).transpose()
gene_to_bioprocess_df = pd.DataFrame(gene_to_bioprocess_results).transpose()
gene_to_pathway_df.to_csv('Dravet_Syndrome_gene_to_pathways_2020_09_14.csv', index = True)
gene_to_bioprocess_df.to_csv('Dravet_Syndrome_gene_to_bioprocesses_2020_09_14.csv', index = True)